In [1]:
import tensorflow as tf

2024-03-22 15:55:46.455384: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-03-22 15:55:46.924121: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-03-22 15:55:48.103477: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [1]:
import tensorflow as tf

print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))
if tf.test.is_built_with_cuda():
    print("TensorFlow was built with CUDA")
else:
    print("TensorFlow was not built with CUDA")

if tf.test.gpu_device_name():
    print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))
else:
    print("Please install GPU version of TF")

2024-03-22 16:31:23.534688: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-03-22 16:31:24.105975: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-03-22 16:31:26.537510: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


Num GPUs Available:  0
TensorFlow was built with CUDA
Please install GPU version of TF


2024-03-22 16:31:29.045329: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-22 16:31:29.270352: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2251] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...
2024-03-22 16:31:29.273684: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-22 16:31:29.273710: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2251] Cannot dlopen some GPU libraries. Please make sure the 

In [4]:
import glob
import os

In [7]:
m="/home/wailmouro/code/benoitfrisque/signlens/training_outputs/model 20240325-120044"
keras_files = glob.glob(os.path.join(m,'model', "*.keras"))
keras_files

[]

In [2]:
!echo $EPOCHS

500


In [1]:
%load_ext autoreload
%autoreload 2

from signlens.preprocessing.data import load_data_subset_csv, unique_train_test_split,filter_out_landmarks,load_relevant_data_subset


In [48]:
train = load_data_subset_csv(balanced=True, random_state=None)

Loading data subset from train_train.csv
    ℹ️ Filtered sequences with missing frames. Size reduced from 77228 to 77228 (100.00%)
    ℹ️ Filtered on n_frames = 100. Size reduced from 77228 to 68935 (89.26%)
    ℹ️ Filtered on n_classes = 250. Size reduced from 68935 to 68935 (100.00%)
    ℹ️ Balanced data, with average of 27.6 elements per class. Size reduced from 68935 to 6893 (10.00%)
✅ Loaded 6893 rows (8.93% of the original 77228 rows) from the dataset.


In [3]:
from signlens.params import *
from signlens.preprocessing.data import load_data_subset_csv, unique_train_test_split,filter_out_landmarks
from signlens.preprocessing.preprocess import preprocess_and_pad_sequences_from_pq_list, encode_labels
from signlens.model.model import initialize_model, compile_model, train_model, evaluate_model
from utils.model_utils import save_results, save_model, load_model, create_model_folder
from sklearn.model_selection import train_test_split

2024-03-26 09:23:50.950735: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-03-26 09:23:50.990481: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-03-26 09:23:51.831637: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [46]:
X_files = train.file_path
y = encode_labels(train.sign)
X_train_files, X_val_files, y_train, y_val = train_test_split(X_files, y, test_size=0.2, stratify=y, random_state=None)

In [7]:
data_columns = ['x', 'y', 'z', 'type','landmark_index']
data=filter_out_landmarks(train.file_path[0], landmark_types_to_remove=[
                                    'face','pose'], data_columns=data_columns)

data[data["type"]=="pose"]


,x,y,z,type,landmark_index
489,0.414427,0.352999,-1.532562,pose,0
490,0.453646,0.296464,-1.466734,pose,1
491,0.480678,0.294072,-1.466874,pose,2
492,0.505012,0.292578,-1.466365,pose,3
493,0.369479,0.304016,-1.445349,pose,4
...,...,...,...,...,...
8654,-0.025172,0.881494,-2.307901,pose,20
8655,0.706267,0.719986,-1.503017,pose,21
8656,-0.000540,0.882726,-2.115515,pose,22
8657,0.653640,1.206206,-0.099809,pose,23


In [53]:
X_train = preprocess_and_pad_sequences_from_pq_list(X_train_files)

100%|██████████| 5514/5514 [00:09<00:00, 586.79it/s]
2024-03-26 09:47:54.348720: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-03-26 09:47:54.349548: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2251] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


In [57]:
test_data = load_data_subset_csv(balanced=True, csv_path=TRAIN_TEST_CSV_PATH, random_state=None)
X_test_files = test_data.file_path
X_test = preprocess_and_pad_sequences_from_pq_list(X_test_files)


Loading data subset from train_test.csv
    ℹ️ Filtered sequences with missing frames. Size reduced from 17233 to 17233 (100.00%)
    ℹ️ Filtered on n_frames = 100. Size reduced from 17233 to 17233 (100.00%)
    ℹ️ Filtered on n_classes = 250. Size reduced from 17233 to 17233 (100.00%)
    ℹ️ Balanced data, with average of 6.9 elements per class. Size reduced from 17233 to 1723 (10.00%)
✅ Loaded 1723 rows (10.00% of the original 17233 rows) from the dataset.


/home/wailmouro/.pyenv/versions/3.10.6/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
  0%|          | 0/1723 [00:00<?, ?it/s]

              x         y         z        type  landmark_index
468         NaN       NaN       NaN   left_hand               0
469         NaN       NaN       NaN   left_hand               1
470         NaN       NaN       NaN   left_hand               2
471         NaN       NaN       NaN   left_hand               3
472         NaN       NaN       NaN   left_hand               4
...         ...       ...       ...         ...             ...
21172  0.385797  0.640931 -0.023434  right_hand              16
21173  0.307752  0.586529 -0.047577  right_hand              17
21174  0.386889  0.605148 -0.047956  right_hand              18
21175  0.397630  0.629893 -0.037001  right_hand              19
21176  0.394772  0.649192 -0.027443  right_hand              20

[2925 rows x 5 columns]              x         y         z        type  landmark_index
468         NaN       NaN       NaN   left_hand               0
469         NaN       NaN       NaN   left_hand               1
470         NaN 

  0%|          | 0/1723 [00:00<?, ?it/s]

             x         y         z        type  landmark_index
468        NaN       NaN       NaN   left_hand               0
469        NaN       NaN       NaN   left_hand               1
470        NaN       NaN       NaN   left_hand               2
471        NaN       NaN       NaN   left_hand               3
472        NaN       NaN       NaN   left_hand               4
...        ...       ...       ...         ...             ...
3253  0.237587  0.483961 -0.123836  right_hand              16
3254  0.169058  0.642791 -0.074188  right_hand              17
3255  0.131039  0.596115 -0.110133  right_hand              18
3256  0.108254  0.564670 -0.127903  right_hand              19
3257  0.093979  0.537315 -0.137195  right_hand              20

[450 rows x 5 columns]             x         y         z        type  landmark_index
468        NaN       NaN       NaN   left_hand               0
469        NaN       NaN       NaN   left_hand               1
470        NaN       NaN       N

An error occurred with multiprocessing: cannot reshape array of size 5772 into shape (25,75,3)
Falling back to sequential processing...
              x         y             z        type  landmark_index
468    0.854094  0.744208 -1.520252e-07   left_hand               0
469    0.832696  0.685398 -1.283050e-02   left_hand               1
470    0.763122  0.638836 -3.132639e-02   left_hand               2
471    0.688207  0.621356 -4.461389e-02   left_hand               3
472    0.633955  0.623305 -5.976132e-02   left_hand               4
...         ...       ...           ...         ...             ...
14113       NaN       NaN           NaN  right_hand              16
14114       NaN       NaN           NaN  right_hand              17
14115       NaN       NaN           NaN  right_hand              18
14116       NaN       NaN           NaN  right_hand              19
14117       NaN       NaN           NaN  right_hand              20

[1950 rows x 5 columns]


ValueError: cannot reshape array of size 5772 into shape (25,75,3)

In [43]:
(N_LANDMARKS_MAX_POSE_TO_TAKE_OFF-N_LANDMARKS_MIN_POSE_TO_TAKE_OFF)

7

In [35]:
(75-8)*16


1072

In [61]:
5475/75

73.0

In [69]:
5402/75

72.02666666666667

In [78]:
load_relevant_data_subset(train.file_path[0])


              x         y             z
468    0.665297  0.648619  3.473000e-07
469    0.661604  0.574149  5.871939e-03
470    0.610408  0.506963 -6.762640e-03
471    0.556034  0.455430 -1.949647e-02
472    0.529652  0.414333 -3.564896e-02
...         ...       ...           ...
39634       NaN       NaN           NaN
39635       NaN       NaN           NaN
39636       NaN       NaN           NaN
39637       NaN       NaN           NaN
39638       NaN       NaN           NaN

[5475 rows x 3 columns]


array([[[ 6.6529685e-01,  6.4861888e-01,  3.4730004e-07],
        [ 6.6160381e-01,  5.7414860e-01,  5.8719395e-03],
        [ 6.1040813e-01,  5.0696325e-01, -6.7626401e-03],
        ...,
        [           nan,            nan,            nan],
        [           nan,            nan,            nan],
        [           nan,            nan,            nan]],

       [[           nan,            nan,            nan],
        [           nan,            nan,            nan],
        [           nan,            nan,            nan],
        ...,
        [           nan,            nan,            nan],
        [           nan,            nan,            nan],
        [           nan,            nan,            nan]],

       [[           nan,            nan,            nan],
        [           nan,            nan,            nan],
        [           nan,            nan,            nan],
        ...,
        [           nan,            nan,            nan],
        [           nan,     

In [77]:
filter_out_landmarks(train.file_path[0],"pose")[20:40]


,frame,row_id,type,landmark_index,x,y,z
488,80,80-left_hand-20,left_hand,20,0.386500,0.575787,-0.060855
489,80,80-pose-0,pose,0,0.473727,0.397591,-1.654494
490,80,80-pose-1,pose,1,0.512779,0.337671,-1.590081
491,80,80-pose-2,pose,2,0.539508,0.337571,-1.590090
492,80,80-pose-3,pose,3,0.564957,0.338097,-1.590144
493,80,80-pose-4,pose,4,0.429495,0.339254,-1.574498
494,80,80-pose-5,pose,5,0.400337,0.339605,-1.573583
495,80,80-pose-6,pose,6,0.373363,0.340466,-1.574177
496,80,80-pose-7,pose,7,0.605847,0.358959,-0.984887
497,80,80-pose-8,pose,8,0.332776,0.363093,-0.874590
